# Нормализация данных и KNN
В этом ноутбуке я рассмотрю различные способы нормализации данных и сравню их качество в связке с методом ближайших соседей. Решается заадача бинарной классификации. 

## Загрузка и предобработка данных
В этом разделе я: 
* загружу датасеты для обучения и тестирования
* заполню пропуски в этих датасетах
* удалю столбцы, отвечающие данным, с которыми будет сложнее работать в рамках использования KNN
* приведу данные к виду, необходимому для корректного запуска классификатора на основе метода ближайших соседей

In [1]:
import pandas
import numpy
import sklearn

In [2]:
ds_train = pandas.read_csv('./train.csv')
ds_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Какие столбцы я удалю?
1. Survived, потому что это и есть результат классификации. Его я сохраню в отдельном векторе, который пригодится при обучении классификатора KNN
2. PassengerId, Ticket и Cabin, потому что эти данные представляют собой некие идентификаторы, уникальные для каждого пассажира. Также с ними сложнее работать из-за того, что они категориальные с большим числом возможных классов.
3. Name, потому что это текстовый признак, с которым пока я не имею опыта работы.

In [3]:
y_train_ds = ds_train['Survived']
ds_train.drop(columns=['Survived'], inplace=True)
ds_train.drop(columns=['PassengerId', 'Ticket', 'Cabin', 'Name'], inplace=True)

In [4]:
ds_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,S
887,1,female,19.0,0,0,30.0000,S
888,3,female,NaN,1,2,23.4500,S
889,1,male,26.0,0,0,30.0000,C


Узнаем, есть ли в полученных данных пропуски:

In [5]:
ds_train.isnull().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

Да, пропуски есть в двух признаках:
1. Age (вещественный). Целых 177 пропусков. Заменю их на моду.
2. Embarked (категориальный). Т.к. у этого признака всего 2 пропуска, то я заменю эти пропуски на самое частое значение этого признака

In [6]:
ds_train['ones'] = 1
ds_train.groupby(['Embarked']).sum()

,Pclass,Age,SibSp,Parch,Fare,ones
Embarked,,,,,,
C,317,4005.92,65,61,10072.2962,168
Q,224,786.50,33,13,1022.2543,77
S,1514,16312.75,368,266,17439.3988,644


Видно, что самое частое значение у признака "Embarked" - "S". Этим значением мы и заполним пропуски:

In [7]:
ds_train['Embarked'] = ds_train['Embarked'].fillna('S')

In [8]:
print(ds_train['Age'].mean(), ds_train['Age'].median(), ds_train['Age'].mode())
train_mode = ds_train['Age'].mode()
ds_train.fillna(value=float(train_mode), inplace=True)

29.69911764705882 28.0 0    24.0
dtype: float64


In [9]:
ds_train.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
ones        0
dtype: int64

Теперь пропущенных значений не осталось. Теперь необходимо заменить категориальные признаки числовыми значениями (кодами этих классов) и конвертировать исходные данные в numpy-матрицы.

In [10]:
ds_train.sample(7)
ds_train["Sex"] = pandas.Categorical(ds_train["Sex"]).codes
ds_train["Embarked"] = pandas.Categorical(ds_train["Embarked"]).codes

In [11]:
ds_train.drop(columns=['ones'], inplace=True)

In [12]:
x_train = ds_train.to_numpy()
y_train = y_train_ds.to_numpy()

In [13]:
x_train

array([[ 3.    ,  1.    , 22.    , ...,  0.    ,  7.25  ,  2.    ],
       [ 1.    ,  0.    , 38.    , ...,  0.    , 71.2833,  0.    ],
       [ 3.    ,  0.    , 26.    , ...,  0.    ,  7.925 ,  2.    ],
       ...,
       [ 3.    ,  0.    , 24.    , ...,  2.    , 23.45  ,  2.    ],
       [ 1.    ,  1.    , 26.    , ...,  0.    , 30.    ,  0.    ],
       [ 3.    ,  1.    , 32.    , ...,  0.    ,  7.75  ,  1.    ]])

Теперь загрузим тестовые данные и проведём с ними такие же манипуляции, как с данными для обучения:

In [14]:
ds_test = pandas.read_csv('./test.csv')

ds_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
ds_test.drop(columns=['PassengerId', 'Ticket', 'Cabin', 'Name'], inplace=True)

In [16]:
ds_test.isnull().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

В тестовом датасете возник неожиданный конфуз - появился пропуск в признаке "Fare". Ну что ж, тоже заменим его на моду

In [17]:
ds_test['Age'].fillna(float(train_mode), inplace=True)
ds_test['Fare'].fillna(float(ds_test['Fare'].mode()), inplace=True)

In [18]:
ds_test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [19]:
ds_test['Sex'] = pandas.Categorical(ds_test['Sex']).codes
ds_test['Embarked'] = pandas.Categorical(ds_test['Embarked']).codes

In [20]:
ds_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [ ]:
x_test = ds_test.to_numpy()

## Обучение классификатора KNN
Обучим реализованный в scikit-learn KNN-классификатор

In [75]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 39, 45, 51, 61, 75, 91, 101]
}

estimator = GridSearchCV(KNeighborsClassifier(), param_grid, scoring=make_scorer(accuracy_score), \
                         verbose=1, return_train_score=True)
estimator.fit(x_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 39, 45, 51, 61,
                                         75, 91, 101]},
             return_train_score=True, scoring=make_scorer(accuracy_score),
             verbose=1)

In [76]:
p_train = estimator.predict(x_train)

In [77]:
accuracy_score(y_train, p_train)

0.7384960718294051

## Нормализация данных
Так как мы используем метрический метод классификации, то для нас важен масштаб признаков. Необходимо добиться того, чтобы признаки были одинакового масштаба. Есть различные способы масштабирования (нормализации) данных:
* Вычистание среднего и деление на стандартное отклонение
* Вычитание минимума и деление на разность максимума и минимума
* Вычитание среднего и деление на разность максимума и минимума
* Предыдущие варианты, где среднее заменено на медиано (она более устойчива к выбросам)

Я рассмотрю методы нормализации/масштабирования, реализованные в sklearn.preprocessing. Проверю, как они влияют на точность полученной модели. Сначала точность на обучающей выборке, а потом и на тестовой (загружу на kaggle метки моделей, обученных на данных, которые были предобработаны различными способами)

In [78]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.preprocessing import QuantileTransformer, RobustScaler, StandardScaler

Обзор тестируемых методов:
1. MaxAbsScaler делит все значения признака на максимальное по модулю значение этого признака. Полученные данные по модулю < 1. Только масштабирует, но не смещает распределение.
2. MinMaxScaler вычитает минимум и делит на разность максимума и минимума. Полученные данные лежат в диапазоне от 0 до 1. Масштабирует и смещает распределение
3. QuantileTransformer тарсформирует распределение признака в нормальное/равномерное распределение (в зависимости от параметров).
4. RobustScaler вычитает медиану и делит на разность 75-й и 25-й квантилей. Устойчив к выбросам не только за счёт медианы, но и за счёт использования 25/75 квантилей вместо минимума/максимума
5. StandardScaler вычитает среднее и делит на стандартное отклонение

Теперь воспользуемся этими методами на практике:

In [79]:
class NormalizedKNN:
    def __init__(self, normalizer, n_neighbors: int = 5):
        self.normalizer = normalizer
        self.n_neighbors = n_neighbors
        self.clf = KNeighborsClassifier(n_neighbors)
    
    def fit(self, x_train, y_train):
        x_train_transformed = self.normalizer.fit_transform(x_train)
        self.clf.fit(x_train_transformed, y_train)
        
    def predict(self, x_test):
        x_test_transformed = self.normalizer.transform(x_test)
        return self.clf.predict(x_test_transformed)    
    
    def get_params(self, deep=True):
        params = {'n_neighbors': self.n_neighbors, 'normalizer': self.normalizer}
        if deep:
            params.update(self.clf.get_params())
        return params
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
            setattr(self.clf, parameter, value)
        return self
    
class Empty:
    def __init__(self):
        pass
    
    def fit_transform(self, data):
        return data
    
    def transform(self, data):
        return data

In [82]:
normalizers = [Empty, MaxAbsScaler, MinMaxScaler,  QuantileTransformer, RobustScaler, StandardScaler]

result = []
for normalizer in normalizers:
    estimator = GridSearchCV(NormalizedKNN(normalizer()), param_grid, scoring=make_scorer(accuracy_score), \
                             verbose=1)
    estimator.fit(x_train[:700], y_train[:700])
    p_train = estimator.predict(x_train)
    result.append((estimator, normalizer.__name__ + ":", accuracy_score(y_train[700:], p_train[700:]), estimator.best_params_))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warning

C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warning

C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warning

C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (560). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "
C:\Users\02031\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (700). n_quantiles is set to n_samples.
  warning

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Fitting 5 folds for each of 24 candidates, totalling 120 fits


In [84]:
for r in result:
    print(*r[1:])

Empty: 0.6701570680628273 {'n_neighbors': 1}
MaxAbsScaler: 0.8272251308900523 {'n_neighbors': 27}
MinMaxScaler: 0.837696335078534 {'n_neighbors': 27}
QuantileTransformer: 0.8219895287958116 {'n_neighbors': 19}
RobustScaler: 0.8272251308900523 {'n_neighbors': 19}
StandardScaler: 0.806282722513089 {'n_neighbors': 7}


Видно, что при сравнении путём кросс-валидации нормализация данных показала значительный прирост в качестве классификации. Теперь нужно применить эти модели к тестовой выборке, загрузить её на kaggle, посмотреть результаты и записать их ниже:
1. Empty (без нормализации): 0.62200
2. MaxAbsScaler: 0.77990
3. MinMaxScaler: 0.78468
4. QuantileTransformer: 0.76794
5. RobustScaler: 0.71531
6. StandardScaler: 0.75837

На данный момент KNN+MinMaxScaler --- это моё лучшее решение на задаче предсказания выживания на титанике. Это решение на момент написание этого текста (21.12.2021) занимает 2394 место в рейтинге (кстати, места вплоть до 1856 имеют то же самое качество). 

In [104]:
for i, r in enumerate(result):
    estimator, normalizer_name, _, _ = r
    p_test = estimator.predict(x_test)
    df_predicted = pandas.DataFrame()
    df_predicted['PassengerId'] = pandas.read_csv('./test.csv')['PassengerId']
    df_predicted['Survived'] = p_test
    print(normalizer_name, type(normalizer_name), len(normalizer_name))
    df_predicted.to_csv(f"res_{i}.csv", index=False)

Empty: <class 'str'> 6
MaxAbsScaler: <class 'str'> 13
MinMaxScaler: <class 'str'> 13
QuantileTransformer: <class 'str'> 20
RobustScaler: <class 'str'> 13
StandardScaler: <class 'str'> 15
